In [9]:
import numpy as np
import pandas as pd

class NMatrixFactorization:
    def __init__(self, num_users, num_items, num_factors=10, learning_rate=0.01, num_epochs=100):
        self.num_users = num_users
        self.num_items = num_items
        self.num_factors = num_factors
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.user_factors = None
        self.item_factors = None
    
    def fit(self, ratings):
        self.ratings = ratings
        
#         self.user_factors = np.random.rand(int(self.num_users), int(self.num_factors))
#         self.item_factors = np.random.rand(int(self.num_items), int(self.num_factors))
        
        self.user_factors = np.ones((int(self.num_users), self.num_factors))
        self.item_factors = np.ones((int(self.num_items), self.num_factors))

        for epoch in range(self.num_epochs):
            for user, item, rating in self.ratings:
                user = int(user)
                item = int(item)
                error = rating - np.dot(self.user_factors[user], self.item_factors[item])
                self.user_factors[user] += self.learning_rate * error * self.item_factors[item]
                self.item_factors[item] += self.learning_rate * error * self.user_factors[user]

                # Ensure non-negativity
                self.user_factors[self.user_factors < 0] = 0
                self.item_factors[self.item_factors < 0] = 0


    def predict(self, user, item):
        user = int(user)
        item = int(item)
        return np.dot(self.user_factors[user], self.item_factors[item])

    def recommend(self, user, top_n=5):
        user_ratings = self.ratings[self.ratings[:, 0] == user]
        user_items = user_ratings[:, 1]
        predictions = np.array([self.predict(user, int(item)) for item in range(int(self.num_items))])
        mask = np.logical_not(np.isin(range(int(self.num_items)), user_items))
        predictions[mask] = -np.inf
        top_items = np.argsort(predictions)[::-1][:top_n]
        return top_items



In [10]:
ratings = pd.read_csv('ratings.csv').drop(['Unnamed: 0'], axis=1)
providers = pd.read_csv('providers.csv').drop(['Unnamed: 0'], axis=1)
r = ratings.to_numpy()

num_users = np.max(r[:, 0]) + 1
num_items = np.max(r[:, 1]) + 1

# Create NMFRecommender instance
nmf = NMatrixFactorization(num_users, num_items, num_factors=10, learning_rate=0.01, num_epochs=100)

# Fit the model
nmf.fit(r)

In [11]:
ratings

,User_id,Registration number,rating
0,0,2119,1.0
1,1,7993,1.0
2,2,2775,1.0
3,3,2839,1.0
4,4,2643,3.0
5,5,2959,3.0
6,6,7430,3.0
7,7,1634,4.0
8,8,8397,5.0
9,9,8995,5.0


In [12]:
import sys
import os

def user2userPredictions(userid, pred_path):
    """
    Сделаем предикт для каждого пользователя и сохраним в файл prediction.csv
    
    :param
        - userid : пользователя id
        - pred_path : куда сохраняем
    """    
    # поиск поставщиков
    reg_num = set(ratings['Registration number'].tolist())
    user = set(ratings[ratings['User_id'] == userid]['Registration number'].tolist())
    diff = list(reg_num - user) 
    
    try:

        # цикл по всем выбраным пользователям для предикта
        for itemid in diff:
            
            # предикт для пользователя, по элементам
            r_hat = nmf.predict(userid, itemid)  
#             print(r_hat)

            # сохраним
            with open(pred_path, 'a+') as file:
                line = '{},{},{}\n'.format(userid, itemid, r_hat)
                file.write(line)
                
    except IndexError:
        pass

def user2userNMF():
    """
    Предикт для всех пользователей, даже с 1 рейтингом   
    """
    # список всех пользователей
    users = ratings['User_id'].unique()
    
    def _progress(count):
        sys.stdout.write('\rRating predictions. Progress status : %.1f%%' % (float(count/len(users))*100.0))
        sys.stdout.flush()
    
    saved_predictions = 'predictionsNMf.csv'    
    if os.path.exists(saved_predictions):
        os.remove(saved_predictions)
    
    with open(saved_predictions, 'a+') as file:
        line = '{},{},{}\n'.format('User_id', 'Registration number', 'predicted_rating')
        file.write(line)
    
    for count, userid in enumerate(users):        
        # делаем предикт
        user2userPredictions(userid, saved_predictions)
        _progress(count)

def user2userRecommendation(userid, N=len(ratings.columns)):
    """
    Делаем предикт для пользователя
    """
    
    saved_predictions = 'predictionsNMf.csv'
    
    predictions = pd.read_csv(saved_predictions, sep=',')
    
    predictions = predictions[predictions['User_id']==userid]
    List = predictions.sort_values(by=['predicted_rating'], ascending=False)[:N]
    
    List = pd.merge(List, providers, on='Registration number', how='inner')
    
    return List

In [13]:
user2userNMF()

Rating predictions. Progress status : 96.0%

In [14]:
# user2userRecommendation(1, 900)

In [15]:
user2userRecommendation(0, 5).drop(['Registration number', 'Предмет поставки', 'Важная информация'], axis=1)

,User_id,predicted_rating,Регистрационный номер,Наименование,Вид деятельности/отрасль,Телефон,Сводный индикатор,"Уставный капитал, RUB",Руководитель - ФИО
0,0,1.936796,1176196042681,"АЛЕКС ФРЕШ, ООО",Торговля оптовая фруктами и овощами,+7 (991) 0855161\n+7 (928) 9001606\n+7 (928) 9...,Низкий риск,10000.0,Баранцев Сергей Александрович
1,0,1.936796,1149204004290,"СЕВТОРГ, ООО",Торговля оптовая напитками,+7 (978) 7167193,Средний риск,700000.0,Кумалагов Заурбек Юрьевич
2,0,1.936796,1121690071030,"ДЕМЕТРА ПОВОЛЖЬЕ, ООО","Торговля оптовая пищевыми продуктами, напиткам...",+7 (917) 2707077,Низкий риск,10000.0,Ибрагимов Ильгам Надирович
3,0,1.936796,1026301504240,"ТОК-ПРОДУКТ, ООО","Торговля оптовая прочими пищевыми продуктами, ...",+7 (846) 9920722\n+7 (84649) 15303\n+7 (927) 2...,Средний риск,10000.0,Косов Константин Алексеевич
4,0,1.936796,1037739627265,"КОРТЛАЙН, ООО",Торговля оптовая мясом и мясными продуктами,+7 (495) 4465695\n+7 (495) 4465698,Низкий риск,10008400.0,Топоровский Владимир Эльевич
